# Análise - FIFA World Cup 2018
-----
# PySpark

### Instalação das bibliotecas

In [53]:
%%sh
pip install spark
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importação das bibliotecas

In [110]:
import pyspark
from pyspark.sql import SparkSession # conexão com o cluster Spark
from pyspark.sql.functions import * # importação das funções do pyspark
from pyspark.sql.types import * # importação dos tipos do pyspark
from pyspark.sql.window import Window # importação das window functions

### Criar / Iniciar Sessão PySpark

In [111]:
spark = (
    SparkSession\
    .builder\
    .master('local')\
    .appName('pyspark_praticando')\
    .getOrCreate()
)

### Criação do Schema

In [112]:
# Exemplo 1:

# 1º parametro: Nome da coluna
# 2º parametro: Tipo de dado
# 3º parametro: True/False para null

wc_schema = StructType([
    StructField('Team', StringType(), True),\
    StructField('#', IntegerType(), True),\
    StructField('Pos', StringType(), True),\
    StructField('FIFA Popular Name', StringType(), True),\
    StructField('Birth Date', StringType(), True),\
    StructField('Shirt Name', StringType(), True),\
    StructField('Club', StringType(), True),\
    StructField('Height', IntegerType(), True),\
    StructField('Weight', IntegerType(), True)
])

In [113]:
# ex2
wc_schema2 = """
            selecao STRING,
            num_camisa INT,
            posicao STRING,
            nome_fifa STRING,
            data_nasc STRING,
            nome_camisa STRING,
            time STRING,
            altura INT,
            peso INT
            """

### Leitura do arquivo

In [114]:
# não é obrigatório a definição de um schema.
# inferSchema para definir um schema automático
# schema=wc_schema
# .option('header','true') -> Esta opção é usada para ler a primeira linha do arquivo CSV como nomes de coluna, por padrão é False

df = spark.read\
     .format('csv')\
     .option('header', 'true')\
     .schema(wc_schema)\
     .load('/content/wc2018-players.csv')
df.show(5)

+---------+---+---+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+---+------------------+----------+----------+--------------------+------+------+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+---+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



### Informações do Schema

In [115]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



#### Alterações dos nomes das colunas

In [116]:
# 1º parametro: nome da coluna
# 2º parametro: nome novo da coluna 

df = df.withColumnRenamed('Team', 'selecao')\
       .withColumnRenamed('#', 'Numero camisa')\
       .withColumnRenamed('Pos', 'Posicao')\
       .withColumnRenamed('FIFA Popular Name', 'Nome Fifa')\
       .withColumnRenamed('Birth Date', 'Data Nascimento')\
       .withColumnRenamed('Shirt Name', 'nome camisa')\
       .withColumnRenamed('Club', 'Time')\
       .withColumnRenamed('Height', 'Altura')\
       .withColumnRenamed('Weight', 'Peso')

df.show(5, truncate=False)

+---------+-------------+-------+------------------+---------------+-----------+------------------------+------+----+
|selecao  |Numero camisa|Posicao|Nome Fifa         |Data Nascimento|nome camisa|Time                    |Altura|Peso|
+---------+-------------+-------+------------------+---------------+-----------+------------------------+------+----+
|Argentina|3            |DF     |TAGLIAFICO Nicolas|31.08.1992     |TAGLIAFICO |AFC Ajax (NED)          |169   |65  |
|Argentina|22           |MF     |PAVON Cristian    |21.01.1996     |PAVÓN      |CA Boca Juniors (ARG)   |169   |65  |
|Argentina|15           |MF     |LANZINI Manuel    |15.02.1993     |LANZINI    |West Ham United FC (ENG)|167   |66  |
|Argentina|18           |DF     |SALVIO Eduardo    |13.07.1990     |SALVIO     |SL Benfica (POR)        |167   |69  |
|Argentina|10           |FW     |MESSI Lionel      |24.06.1987     |MESSI      |FC Barcelona (ESP)      |170   |72  |
+---------+-------------+-------+------------------+----

### Alteração Coluna - Substituição do espaço por _ <br>e letra maiuscula por minuscula

In [118]:
df = df.select([col(nome).alias(nome.replace(' ', '_').replace(nome, nome.lower())) for nome in df.columns])
df.show(1)

+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|          time|altura|peso|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     31.08.1992| TAGLIAFICO|AFC Ajax (NED)|   169|  65|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+
only showing top 1 row



### Criação de novas colunas

In [119]:
# Função lambda para dividir a data em dia, mes e ano

# udf: Funções definidas pelo usuário
# udf(): Este método usará a função lambda para fazer um loop sobre os dados, 
# e seu argumento aceitará a função lambda, e o valor lambda se tornará um argumento 
# para a função que queremos fazer como UDF. 

dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

In [120]:
# Adicionando as novas colunas
df = df.withColumn('dia', dia('data_nascimento'))\
       .withColumn('mes', mes('data_nascimento'))\
       .withColumn('ano', ano('data_nascimento'))
df.show(1)

+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|          time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     31.08.1992| TAGLIAFICO|AFC Ajax (NED)|   169|  65| 31| 08|1992|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
only showing top 1 row



In [121]:
# lit() e typedLit() são usadas para adicionar uma nova coluna ao DataFrame atribuindo um valor literal ou constante. 
# Ambas as funções retornam o tipo de coluna como tipo de retorno.
df.withColumn('alt_peso', lit(col('altura') - col('peso'))).show(5)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+--------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|alt_peso|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+--------+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     104|
|Argentina|           22|     MF|    PAVON Cristian|     21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     104|
|Argentina|           15|     MF|    LANZINI Manuel|     15.02.1993|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     101|
|Argentina|           18|     DF|    SALVIO Eduardo|     13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|      98|
|Argentina|           10|     FW|      MESSI Lio

In [122]:
# coluna condicional
df.withColumn('sub', substring('selecao', -3, 3)).show(5)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+---+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|sub|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+---+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|ina|
|Argentina|           22|     MF|    PAVON Cristian|     21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|ina|
|Argentina|           15|     MF|    LANZINI Manuel|     15.02.1993|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|ina|
|Argentina|           18|     DF|    SALVIO Eduardo|     13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|ina|
|Argentina|           10|     FW|      MESSI Lionel|     24.06.1987|      MESSI|  F

### Concatenação e criação de uma nova coluna

In [123]:
df = df.withColumn('data_nascimento', concat_ws('-', 'ano', 'mes', 'dia').cast(DateType()))
df.show(1)

+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|          time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|AFC Ajax (NED)|   169|  65| 31| 08|1992|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
only showing top 1 row



### Exclusão de coluna

In [124]:
df = df.drop('data_nasc')
df.show(1)

+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|          time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|AFC Ajax (NED)|   169|  65| 31| 08|1992|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
only showing top 1 row



### Simples backup

In [125]:
df2 = df

### Seleção de colunas

In [126]:
# Exemplo 1 de seleção
df.select('selecao', 'nome_camisa', 'numero_camisa').show(1)

+---------+-----------+-------------+
|  selecao|nome_camisa|numero_camisa|
+---------+-----------+-------------+
|Argentina| TAGLIAFICO|            3|
+---------+-----------+-------------+
only showing top 1 row



In [127]:
# Exemplo 2 de seleção
df.select(col('nome_fifa'), col('selecao'), col('time')).show(1)

+------------------+---------+--------------+
|         nome_fifa|  selecao|          time|
+------------------+---------+--------------+
|TAGLIAFICO Nicolas|Argentina|AFC Ajax (NED)|
+------------------+---------+--------------+
only showing top 1 row



In [128]:
# Exemplo 3 de seleção
df.select(df['posicao'], df['nome_fifa']).show(1)

+-------+------------------+
|posicao|         nome_fifa|
+-------+------------------+
|     DF|TAGLIAFICO Nicolas|
+-------+------------------+
only showing top 1 row



In [129]:
# Exemplo 4 de seleção
df.select('nome_fifa selecao time'.split()).show(2)

+------------------+---------+--------------------+
|         nome_fifa|  selecao|                time|
+------------------+---------+--------------------+
|TAGLIAFICO Nicolas|Argentina|      AFC Ajax (NED)|
|    PAVON Cristian|Argentina|CA Boca Juniors (...|
+------------------+---------+--------------------+
only showing top 2 rows



In [130]:
# Exemplo 5 de seleção
df.select(col('selecao').alias('selecao_copa'), 
          col('nome_fifa').alias('nome')
         ).show(2)

+------------+------------------+
|selecao_copa|              nome|
+------------+------------------+
|   Argentina|TAGLIAFICO Nicolas|
|   Argentina|    PAVON Cristian|
+------------+------------------+
only showing top 2 rows



### Filtrando DataFrame do PySpark

In [131]:
# Retorna somente os dados da seleção brasileira
# exemplo 1
df.filter("selecao = 'Brazil'").show(2)

+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           18|     MF|     FRED|     1993-03-05|       FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|
| Brazil|           21|     FW|   TAISON|     1988-01-13|     TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
only showing top 2 rows



In [132]:
# Retorna somente os dados da seleção brasileira
# exemplo 2
df.filter(col('selecao') == 'Brazil').show(2)

+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           18|     MF|     FRED|     1993-03-05|       FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|
| Brazil|           21|     FW|   TAISON|     1988-01-13|     TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
only showing top 2 rows



### Filtrar DataFrame do PySpark com AND (&)

In [133]:
# AND com 2 condições
df.filter((col('selecao') == 'Argentina') & 
          (col('altura') > 169)).show(5)     

+---------+-------------+-------+-----------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|        nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+-----------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|           10|     FW|     MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|
|Argentina|            4|     DF| ANSALDI Cristian|     1986-09-20|    ANSALDI|     Torino FC (ITA)|   181|  73| 20| 09|1986|
|Argentina|            5|     MF|     BIGLIA Lucas|     1986-01-30|     BIGLIA|      AC Milan (ITA)|   175|  73| 30| 01|1986|
|Argentina|            7|     MF|      BANEGA Ever|     1988-06-29|     BANEGA|    Sevilla FC (ESP)|   175|  73| 29| 06|1988|
|Argentina|           14|     DF|MASCHERANO Javier|     1984-06-08| MASCHERANO|Hebei China Fortu...|   174|  73| 08| 0

In [134]:
df.filter((col('selecao') == 'Brazil') & 
          (col('peso') > 80)).show(5)

+-------+-------------+-------+--------------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|     nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+--------------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           15|     MF|      PAULINHO|     1988-07-25|   PAULINHO|  FC Barcelona (ESP)|   181|  81| 25| 07|1988|
| Brazil|            4|     DF| PEDRO GEROMEL|     1985-09-21|    GEROMEL|   Grêmio FBPA (BRA)|   190|  84| 21| 09|1985|
| Brazil|            5|     MF|      CASEMIRO|     1992-02-23|   CASEMIRO|Real Madrid CF (ESP)|   185|  84| 23| 02|1992|
| Brazil|            8|     MF|RENATO AUGUSTO|     1988-02-08| R. AUGUSTO| Beijing Guoan (CHN)|   186|  86| 08| 02|1988|
| Brazil|           23|     GK|       EDERSON|     1993-08-17|    EDERSON|Manchester City F...|   188|  86| 17| 08|1993|
+-------+-------------+-------+-

In [135]:
# outra forma
df.filter("selecao = 'Brazil'")\
  .filter("peso > 80").show(5)

+-------+-------------+-------+--------------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|     nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+--------------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           15|     MF|      PAULINHO|     1988-07-25|   PAULINHO|  FC Barcelona (ESP)|   181|  81| 25| 07|1988|
| Brazil|            4|     DF| PEDRO GEROMEL|     1985-09-21|    GEROMEL|   Grêmio FBPA (BRA)|   190|  84| 21| 09|1985|
| Brazil|            5|     MF|      CASEMIRO|     1992-02-23|   CASEMIRO|Real Madrid CF (ESP)|   185|  84| 23| 02|1992|
| Brazil|            8|     MF|RENATO AUGUSTO|     1988-02-08| R. AUGUSTO| Beijing Guoan (CHN)|   186|  86| 08| 02|1988|
| Brazil|           23|     GK|       EDERSON|     1993-08-17|    EDERSON|Manchester City F...|   188|  86| 17| 08|1993|
+-------+-------------+-------+-

### Filtrar DataFrame do PySpark com OR (|)

In [136]:
df.filter(
    (col('nome_fifa') == 'MESSI Lionel') |
    (col('nome_fifa') == 'CRISTIANO RONALDO') |
    (col('nome_fifa') == 'NEYMAR') |
    (col('altura') == 190)
).show(5)

+---------+-------------+-------+---------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|      nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+---------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|           10|     FW|   MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|
|Australia|            9|     FW|     JURIC Tomi|     1991-07-22|      JURIC|     FC Luzern (SUI)|   190|  80| 22| 07|1991|
|Australia|            6|     DF| JURMAN Matthew|     1989-12-08|     JURMAN|Suwon Samsung Blu...|   190|  83| 08| 12|1989|
|  Belgium|            4|     DF|KOMPANY Vincent|     1986-04-10|    KOMPANY|Manchester City F...|   190|  85| 10| 04|1986|
|  Belgium|           15|     DF| MEUNIER Thomas|     1991-09-12|    MEUNIER|Paris Saint-Germa...|   190|  88| 12| 09|1991|
+-------

### Filtrar DataFrame do PySpark com AND(&) e OR(|) 

In [137]:
df.filter(
    (col('selecao') == 'Brazil') &
    (col('posicao') == 'DF') |
    (col('nome_fifa') == 'MESSI Lionel') 
).show(5)

+---------+-------------+-------+------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|   nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|           10|     FW|MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|
|   Brazil|           22|     DF|      FAGNER|     1989-06-11|     FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|
|   Brazil|            6|     DF| FILIPE LUIS|     1985-08-09|FILIPE LUIS|Atletico Madrid (...|   182|  73| 09| 08|1985|
|   Brazil|           13|     DF|  MARQUINHOS|     1994-05-14| MARQUINHOS|Paris Saint-Germa...|   183|  75| 14| 05|1994|
|   Brazil|            3|     DF|     MIRANDA|     1984-09-07|    MIRANDA|FC Internazionale...|   186|  78| 07| 09|1984|
+---------+-------------+-------

In [138]:
df.filter(
    (col('selecao') == 'Brazil') &
    (col('posicao') == 'DF') |
    (col('altura') > 190) &
    (col('selecao') == 'Belgium')
).show()

+-------+-------------+-------+-----------------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|        nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+-----------------+---------------+-----------+--------------------+------+----+---+---+----+
|Belgium|            8|     MF|FELLAINI Marouane|     1987-11-22|   FELLAINI|Manchester United...|   194|  85| 22| 11|1987|
|Belgium|           13|     GK|    CASTEELS Koen|     1992-06-25|   CASTEELS| VfL Wolfsburg (GER)|   197|  86| 25| 06|1992|
|Belgium|           12|     GK|   MIGNOLET Simon|     1988-03-06|   MIGNOLET|  Liverpool FC (ENG)|   193|  87| 06| 03|1988|
|Belgium|            1|     GK| COURTOIS Thibaut|     1992-05-11|   COURTOIS|    Chelsea FC (ENG)|   199|  91| 11| 05|1992|
| Brazil|           22|     DF|           FAGNER|     1989-06-11|     FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|
| Brazil

# Window Function 

### row_number() - Número de linhas 

In [139]:
# contagem por seleção, ordenado pela altura
num_linha = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('cont_selecao', row_number().over(num_linha)).show(30)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+------------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|cont_selecao|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+------------+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|           1|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|           2|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|           3|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|           4|
|Argentina|         

### rank()

In [140]:
# jogadores com a mesma altura, fica com o mesmo ranking.
# além disso, ele pula ranking
# ex: 1, 2, 3, 3, 5, 6, 7, 7, 7, 10
num_rank1 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('rank1', rank().over(num_rank1)).show()

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+-----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|rank1|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+-----+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|    1|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|    2|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|    3|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|    3|
|Argentina|           23|     GK|CABALLERO Wilfredo|     1981-09-28| 

### dense_rank()

In [141]:
# cria um rank sem pular números
# ex: 1, 2, 3, 4...
num_rank2 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('dense_rank', dense_rank().over(num_rank2)).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|dense_rank|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----------+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|         1|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|         2|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|         3|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|         3|
|Argentina|           23|     GK|C

### percent_rank()

In [142]:
# Calcula a classificação percentual de dada linha (0 e 1).
# O valor percentual é calculado por 
# (classificação determinada pela função RANK - 1) / (número de linhas de destino - 1) 
# e a primeira linha é sempre 0.

porcentagem = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('%', percent_rank().over(porcentagem)).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+--------------------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|                   %|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+--------------------+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|                 0.0|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|0.045454545454545456|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990| 0.09090909090909091|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85

### ntile()

In [143]:
# A função de janela NTILE divide as linhas ordenadas na partição no número especificado de 
# grupos classificados de tamanho o mais igual possível e 
# retorna o grupo em que dada linha se encontra.
# ntile() -> recebe como parametro a qtd de grupos q deseja criar
parte = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('partes', ntile(5).over(parte)).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|partes|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+------+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|     1|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1|
|Argentina|           23|     GK|CABALLERO Wilfredo|     1981-

### lag()

In [144]:
# A função LAG retorna os valores para uma linha em determinado 
# deslocamento acima (antes) da linha atual na partição.
# lag() recebe como parametro aonde ele vai fazer o degrau
# cria uma coluna nova baseada no peso anterior
degrau = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('lag', lag('peso').over(degrau)).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano| lag|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|null|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|  85|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|  90|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|  82|
|Argentina|           23|     GK|CABALLERO Wilfredo|     1981-09-28|  CABALL

### lead()

In [145]:
degrau = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('lead', lead('peso').over(degrau)).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|lead|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+----+
|Argentina|            6|     DF|    FAZIO Federico|     1987-03-17|      FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|  90|
|Argentina|            1|     GK|     GUZMAN Nahuel|     1986-02-10|     GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|  82|
|Argentina|           16|     DF|       ROJO Marcos|     1990-03-20|       ROJO|Manchester United...|   189|  82| 20| 03|1990|  85|
|Argentina|           12|     GK|     ARMANI Franco|     1986-10-16|     ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|  80|
|Argentina|           23|     GK|CABALLERO Wilfredo|     1981-09-28|  CABALL

# AGREGAÇÕES

### GroupBy + AGG 1

In [146]:
# Média da altura de cada seleção
# https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/
df.groupBy('selecao')\
  .agg({'altura': 'avg'})\
  .orderBy('avg(altura)', ascending=False)\
  .show()

+--------------+------------------+
|       selecao|       avg(altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
+--------------+------------------+
only showing top 20 rows



### GroupBy + AGG 2

In [147]:
df.groupBy('selecao')\
  .agg(avg('altura'))\
  .orderBy('avg(altura)', ascending=False)\
  .show()

+--------------+------------------+
|       selecao|       avg(altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
+--------------+------------------+
only showing top 20 rows



# Manipulações

### where

In [148]:
# é a mesma coisa que o filter
df.where("selecao = 'Brazil'")\
  .where("posicao = 'DF'").show()

+-------+-------------+-------+-------------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|    nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+-------------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           22|     DF|       FAGNER|     1989-06-11|     FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|
| Brazil|            6|     DF|  FILIPE LUIS|     1985-08-09|FILIPE LUIS|Atletico Madrid (...|   182|  73| 09| 08|1985|
| Brazil|           13|     DF|   MARQUINHOS|     1994-05-14| MARQUINHOS|Paris Saint-Germa...|   183|  75| 14| 05|1994|
| Brazil|            3|     DF|      MIRANDA|     1984-09-07|    MIRANDA|FC Internazionale...|   186|  78| 07| 09|1984|
| Brazil|           14|     DF|       DANILO|     1991-07-15|     DANILO|Manchester City F...|   184|  78| 15| 07|1991|
| Brazil|            2|     DF| THIAGO S

### describe()

In [149]:
df.describe().show()

+-------+---------+-----------------+-------+------------+-----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|  selecao|    numero_camisa|posicao|   nome_fifa|nome_camisa|                time|           altura|             peso|               dia|               mes|               ano|
+-------+---------+-----------------+-------+------------+-----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|    736|         736|        736|                 736|              736|              736|               736|               736|               736|
|   mean|     null|             12.0|   null|        null|       null|                null|182.4076086956522|77.18885869565217|15.793478260869565|5.8790760869565215| 1990.110054347826|
| stddev|     null|6.637760461599851|   null|        null|       null|     

In [150]:
df.where("selecao = 'Brazil'").describe().show()

+-------+-------+-----------------+-------+---------+-----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|selecao|    numero_camisa|posicao|nome_fifa|nome_camisa|                time|           altura|             peso|              dia|              mes|              ano|
+-------+-------+-----------------+-------+---------+-----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|     23|               23|     23|       23|         23|                  23|               23|               23|               23|               23|               23|
|   mean|   null|             12.0|   null|     null|       null|                null|180.7826086956522|76.56521739130434|11.26086956521739|6.130434782608695|1989.391304347826|
| stddev|   null|6.782329983125267|   null|     null|       null|                null|7.354383490255254|8.239737898

### distinct()

In [151]:
# retorna os valores não repetidos
df.select(col('selecao')).distinct().show()

+--------------+
|       selecao|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
+--------------+
only showing top 20 rows



### collect()

In [152]:
# retorna em formato de lista
lista = df.select(col('selecao')).distinct().collect()
lista

[Row(selecao='Russia'),
 Row(selecao='Senegal'),
 Row(selecao='Sweden'),
 Row(selecao='IR Iran'),
 Row(selecao='Germany'),
 Row(selecao='France'),
 Row(selecao='Argentina'),
 Row(selecao='Belgium'),
 Row(selecao='Peru'),
 Row(selecao='Croatia'),
 Row(selecao='Nigeria'),
 Row(selecao='Korea Republic'),
 Row(selecao='Spain'),
 Row(selecao='Denmark'),
 Row(selecao='Morocco'),
 Row(selecao='Panama'),
 Row(selecao='Iceland'),
 Row(selecao='Uruguay'),
 Row(selecao='Mexico'),
 Row(selecao='Tunisia'),
 Row(selecao='Saudi Arabia'),
 Row(selecao='Switzerland'),
 Row(selecao='Brazil'),
 Row(selecao='Japan'),
 Row(selecao='England'),
 Row(selecao='Poland'),
 Row(selecao='Portugal'),
 Row(selecao='Australia'),
 Row(selecao='Costa Rica'),
 Row(selecao='Egypt'),
 Row(selecao='Serbia'),
 Row(selecao='Colombia')]

In [153]:
paises = [pais[0] for pais in lista]
paises

['Russia',
 'Senegal',
 'Sweden',
 'IR Iran',
 'Germany',
 'France',
 'Argentina',
 'Belgium',
 'Peru',
 'Croatia',
 'Nigeria',
 'Korea Republic',
 'Spain',
 'Denmark',
 'Morocco',
 'Panama',
 'Iceland',
 'Uruguay',
 'Mexico',
 'Tunisia',
 'Saudi Arabia',
 'Switzerland',
 'Brazil',
 'Japan',
 'England',
 'Poland',
 'Portugal',
 'Australia',
 'Costa Rica',
 'Egypt',
 'Serbia',
 'Colombia']

### when() / otherwhise()
- if / else

In [154]:
df.withColumn(
    'if_else',\
    when(col('selecao') == 'Argentina', 'manitos')\
    .otherwise('outros')
).show(25)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+-------+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|if_else|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+-------+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|manitos|
|Argentina|           22|     MF|    PAVON Cristian|     1996-01-21|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|manitos|
|Argentina|           15|     MF|    LANZINI Manuel|     1993-02-15|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|manitos|
|Argentina|           18|     DF|    SALVIO Eduardo|     1990-07-13|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|manitos|
|Argentina|           10|     FW|      MESSI Lionel|   

### concat()


In [155]:
df_brazil = df.filter("selecao = 'Brazil'")
df_brazil.show(1)
print(df_brazil.count())

+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           18|     MF|     FRED|     1993-03-05|       FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|
+-------+-------------+-------+---------+---------------+-----------+--------------------+------+----+---+---+----+
only showing top 1 row

23


In [156]:
df_argentina = df.filter("selecao = 'Argentina'")
df_argentina.show(1)
print(df_argentina.count())

+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|          time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|AFC Ajax (NED)|   169|  65| 31| 08|1992|
+---------+-------------+-------+------------------+---------------+-----------+--------------+------+----+---+---+----+
only showing top 1 row

23


In [157]:
df_bra_arg = df_brazil.union(df_argentina)
df_bra_arg.show(50)
print(df_bra_arg.count())

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|   Brazil|           18|     MF|              FRED|     1993-03-05|       FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|
|   Brazil|           21|     FW|            TAISON|     1988-01-13|     TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|
|   Brazil|           17|     MF|       FERNANDINHO|     1985-05-04|FERNANDINHO|Manchester City F...|   179|  67| 04| 05|1985|
|   Brazil|           22|     DF|            FAGNER|     1989-06-11|     FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|
|   Brazil|           10|     FW|            NEYMAR|     1992-02-05|  NEYMAR JR|Paris Saint-Germa...|   175|  6

# JOINS

In [158]:
df_arg = df.filter('Selecao = "Argentina"')
df_bra = df.filter('Selecao = "Brazil"')

In [159]:
# Dropando algumas cols. para uma melhor visualização 
df_arg = df_arg.drop('time', 'dia', 'mes', 'ano', 'peso', 'data_nascimento')
df_bra = df_bra.drop('time', 'dia', 'mes', 'ano', 'peso', 'data_nascimento')

In [160]:
df_bra = df_bra.withColumn("numero_camisa", when(df_bra.numero_camisa == 23, 80).otherwise(df_bra.numero_camisa))
df_arg = df_arg.withColumn("numero_camisa", when(df_arg.numero_camisa == 6, 99).otherwise(df_arg.numero_camisa))

### inner join
- retorna os dados que tem correspondências

In [161]:
df_arg.join(
    df_bra, 
    df_arg['numero_camisa'] == df_bra['numero_camisa'], 
    'inner'
).show(25)

+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|selecao|numero_camisa|posicao|        nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|Argentina|            3|     DF|TAGLIAFICO Nicolas| TAGLIAFICO|   169| Brazil|            3|     DF|          MIRANDA|    MIRANDA|   186|
|Argentina|           22|     MF|    PAVON Cristian|      PAVÓN|   169| Brazil|           22|     DF|           FAGNER|     FAGNER|   168|
|Argentina|           15|     MF|    LANZINI Manuel|    LANZINI|   167| Brazil|           15|     MF|         PAULINHO|   PAULINHO|   181|
|Argentina|           18|     DF|    SALVIO Eduardo|     SALVIO|   167| Brazil|           18|     MF|             FRED|       FRED|   169|
|Argentina|           10|  

### left join
- retorna todas as linhas do df da esquerda, tendo ou não correspondencia com o df da direita

In [162]:
df_novo = df_arg.join(df_bra, df_arg['numero_camisa'] == df_bra['numero_camisa'], 'left')
df_novo.show(25)

+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|selecao|numero_camisa|posicao|        nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|Argentina|            3|     DF|TAGLIAFICO Nicolas| TAGLIAFICO|   169| Brazil|            3|     DF|          MIRANDA|    MIRANDA|   186|
|Argentina|           22|     MF|    PAVON Cristian|      PAVÓN|   169| Brazil|           22|     DF|           FAGNER|     FAGNER|   168|
|Argentina|           15|     MF|    LANZINI Manuel|    LANZINI|   167| Brazil|           15|     MF|         PAULINHO|   PAULINHO|   181|
|Argentina|           18|     DF|    SALVIO Eduardo|     SALVIO|   167| Brazil|           18|     MF|             FRED|       FRED|   169|
|Argentina|           10|  

### right join
- retorna todas as linhas do df da direita, tendo ou não correspondencia com o df da esquerda

In [163]:
df_arg.join(
    df_bra, 
    df_arg['numero_camisa'] == df_bra['numero_camisa'], 
    'right'
).show(25)


+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|selecao|numero_camisa|posicao|        nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|Argentina|           18|     DF|    SALVIO Eduardo|     SALVIO|   167| Brazil|           18|     MF|             FRED|       FRED|   169|
|Argentina|           21|     FW|      DYBALA Paulo|     DYBALA|   177| Brazil|           21|     FW|           TAISON|     TAISON|   172|
|Argentina|           17|     DF|  OTAMENDI Nicolas|   OTAMENDI|   181| Brazil|           17|     MF|      FERNANDINHO|FERNANDINHO|   179|
|Argentina|           22|     MF|    PAVON Cristian|      PAVÓN|   169| Brazil|           22|     DF|           FAGNER|     FAGNER|   168|
|Argentina|           10|  

### full join
- junção do left + right join

In [164]:
df_arg.join(
    df_bra, 
    df_arg['numero_camisa'] == df_bra['numero_camisa'], 
    'full'
).show(25)

+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|selecao|numero_camisa|posicao|        nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+-------+-------------+-------+-----------------+-----------+------+
|Argentina|            1|     GK|     GUZMAN Nahuel|     GUZMÁN|   192| Brazil|            1|     GK|          ALISSON|  A. BECKER|   193|
|Argentina|            2|     DF|   MERCADO Gabriel|    MERCADO|   181| Brazil|            2|     DF|     THIAGO SILVA|   T. SILVA|   183|
|Argentina|            3|     DF|TAGLIAFICO Nicolas| TAGLIAFICO|   169| Brazil|            3|     DF|          MIRANDA|    MIRANDA|   186|
|Argentina|            4|     DF|  ANSALDI Cristian|    ANSALDI|   181| Brazil|            4|     DF|    PEDRO GEROMEL|    GEROMEL|   190|
|Argentina|            5|  

### semi join
- similar ao inner join, porém apenas os dados do df esquerdo é mostrado

In [165]:
df_arg.join(
    df_bra, 
    df_arg['numero_camisa'] == df_bra['numero_camisa'], 
    'semi'
).show(25)

+---------+-------------+-------+------------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+
|Argentina|            3|     DF|TAGLIAFICO Nicolas| TAGLIAFICO|   169|
|Argentina|           22|     MF|    PAVON Cristian|      PAVÓN|   169|
|Argentina|           15|     MF|    LANZINI Manuel|    LANZINI|   167|
|Argentina|           18|     DF|    SALVIO Eduardo|     SALVIO|   167|
|Argentina|           10|     FW|      MESSI Lionel|      MESSI|   170|
|Argentina|            4|     DF|  ANSALDI Cristian|    ANSALDI|   181|
|Argentina|            5|     MF|      BIGLIA Lucas|     BIGLIA|   175|
|Argentina|            7|     MF|       BANEGA Ever|     BANEGA|   175|
|Argentina|           14|     DF| MASCHERANO Javier| MASCHERANO|   174|
|Argentina|           21|     FW|      DYBALA Paulo|     DYBALA|   177|
|Argentina|           19|     FW|     AGUERO Sergio|     AGÜERO|

### anti join
- mostra os dados do df do lado esquerdo não possuem correspondencias

In [166]:
df_arg.join(
    df_bra, 
    df_arg['numero_camisa'] == df_bra['numero_camisa'], 
    'anti'
).show(25)

+---------+-------------+-------+------------------+-----------+------+
|  selecao|numero_camisa|posicao|         nome_fifa|nome_camisa|altura|
+---------+-------------+-------+------------------+-----------+------+
|Argentina|           23|     GK|CABALLERO Wilfredo|  CABALLERO|   186|
|Argentina|           99|     DF|    FAZIO Federico|      FAZIO|   199|
+---------+-------------+-------+------------------+-----------+------+



# Spark SQL

### Criando um banco de dados

In [167]:
spark.sql('CREATE DATABASE wc_2018').show()

++
||
++
++



### Conectando-se no banco

In [168]:
spark.sql('USE wc_2018').show()

++
||
++
++



### Transformando o DataFrame do PySpark em uma tabela

In [169]:
# df.saveAsTable('myTable') -> Salva a tabela no armazenamento (HDFS/S3)
# df.createOrReplaceTempView("my_temp_table") -> É apenas um identificador a ser usado para o DAG de df. 
# Nada é realmente armazenado na memória ou no disco.
df.write.saveAsTable('wc_players')

### Verificando se a tabela foi criada

In [170]:
# select na tabela wc_players
spark.sql('SHOW TABLES').show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  wc_2018|wc_players|      false|
+---------+----------+-----------+



In [171]:
spark.sql('SELECT * FROM wc_players').show(5)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|
|Argentina|           22|     MF|    PAVON Cristian|     1996-01-21|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|
|Argentina|           15|     MF|    LANZINI Manuel|     1993-02-15|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|
|Argentina|           18|     DF|    SALVIO Eduardo|     1990-07-13|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|
|Argentina|           10|     FW|      MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  7

### Criando e salvando um arquivo Parquet

In [172]:
df.write.format('parquet').save('/content/wc_players_parquet')

### Criando uma tabela externa

In [173]:
df.write.option('path', '/content/wc_players_parquet/part-00000-f98f9150-2ec6-4ad8-8ae3-c79219415a4f-c000.snappy.parquet').saveAsTable('wc_players_parquet')

In [174]:
spark.sql('SELECT * FROM wc_players_parquet').show(5)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|
|Argentina|           22|     MF|    PAVON Cristian|     1996-01-21|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|
|Argentina|           15|     MF|    LANZINI Manuel|     1993-02-15|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|
|Argentina|           18|     DF|    SALVIO Eduardo|     1990-07-13|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|
|Argentina|           10|     FW|      MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  7

### Verificando se existe um caminho fisico da tabela
- Se existir significa que ela é uma tabela externa (não gerenciada)
- Se não existir significa que ela é uma tabela gerenciada

In [175]:
spark.sql('SHOW CREATE TABLE wc_players').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE wc_2018.wc_players (\n  selecao STRING,\n  numero_camisa INT,\n  posicao STRING,\n  nome_fifa STRING,\n  data_nascimento DATE,\n  nome_camisa STRING,\n  time STRING,\n  altu

In [176]:
# externa pq tem um local
spark.sql('SHOW CREATE TABLE wc_players_parquet').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Retorna o catalogo da tabela do banco de dados
- name
- database
- description
- tableType
- isTemporary

In [177]:
spark.catalog.listTables()

[Table(name='wc_players', database='wc_2018', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='wc_players_parquet', database='wc_2018', description=None, tableType='EXTERNAL', isTemporary=False)]

### Views

In [178]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW wc_players_view AS
    SELECT * FROM wc_players
    WHERE selecao = "Brazil"
""").show()

++
||
++
++



In [179]:
spark.sql('SELECT * FROM wc_players_view').show(5)

+-------+-------------+-------+-----------+---------------+-----------+--------------------+------+----+---+---+----+
|selecao|numero_camisa|posicao|  nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+-------+-------------+-------+-----------+---------------+-----------+--------------------+------+----+---+---+----+
| Brazil|           18|     MF|       FRED|     1993-03-05|       FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|
| Brazil|           21|     FW|     TAISON|     1988-01-13|     TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|
| Brazil|           17|     MF|FERNANDINHO|     1985-05-04|FERNANDINHO|Manchester City F...|   179|  67| 04| 05|1985|
| Brazil|           22|     DF|     FAGNER|     1989-06-11|     FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|
| Brazil|           10|     FW|     NEYMAR|     1992-02-05|  NEYMAR JR|Paris Saint-Germa...|   175|  68| 05| 02|1992|
+-------+-------------+-------+-----------+-------------

In [180]:
# VIEW GLOBAL
spark.sql("""
    CREATE OR REPLACE GLOBAL TEMP VIEW wc_players_view_global AS
    SELECT * FROM wc_players
    WHERE selecao = 'Argentina'
""").show()

++
||
++
++



In [181]:
spark.sql('SELECT * FROM global_temp.wc_players_view_global').show(5)

+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|  selecao|numero_camisa|posicao|         nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|dia|mes| ano|
+---------+-------------+-------+------------------+---------------+-----------+--------------------+------+----+---+---+----+
|Argentina|            3|     DF|TAGLIAFICO Nicolas|     1992-08-31| TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|
|Argentina|           22|     MF|    PAVON Cristian|     1996-01-21|      PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|
|Argentina|           15|     MF|    LANZINI Manuel|     1993-02-15|    LANZINI|West Ham United F...|   167|  66| 15| 02|1993|
|Argentina|           18|     DF|    SALVIO Eduardo|     1990-07-13|     SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|
|Argentina|           10|     FW|      MESSI Lionel|     1987-06-24|      MESSI|  FC Barcelona (ESP)|   170|  7

In [182]:
spark.sql(
    """
        SELECT selecao, 
        ROUND(AVG(2022 - ano), 2) as media_idade_selecao,
        MAX(altura) as altura_max_selecao
        FROM wc_players
        GROUP BY selecao
        ORDER BY media_idade_selecao DESC
    """
).show()

+--------------+-------------------+------------------+
|       selecao|media_idade_selecao|altura_max_selecao|
+--------------+-------------------+------------------+
|    Costa Rica|              33.52|               196|
|        Mexico|               33.3|               190|
|        Panama|              33.26|               197|
|     Argentina|              33.22|               199|
|         Egypt|              32.96|               194|
|        Russia|              32.91|               196|
|  Saudi Arabia|              32.78|               192|
|        Brazil|              32.61|               195|
|       Iceland|              32.52|               198|
|         Japan|              32.52|               189|
|      Portugal|              32.43|               191|
|         Spain|              32.43|               194|
|        Poland|              32.26|               195|
|     Australia|              32.22|               193|
|        Sweden|              32.17|            